In [ ]:
#final pagination with print tasks

import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException

print("Setting up the Selenium WebDriver...")
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

print("Navigating to the initial URL...")
url = "https://www.casey.vic.gov.au/view-planning-applications"
driver.get(url)

print("Clicking the necessary button to get to the search page...")
button_xpath = '//*[@id="block-content"]/article/div/div/div/p[8]/a'
button = wait.until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
button.click()

print("Entering the desired date range...")
date_from_xpath = '//*[@id="ctl00_Content_txtDateFrom_txtText"]'
date_to_xpath = '//*[@id="ctl00_Content_txtDateTo_txtText"]'
date_from = wait.until(EC.element_to_be_clickable((By.XPATH, date_from_xpath)))
date_to = wait.until(EC.element_to_be_clickable((By.XPATH, date_to_xpath)))

date_from.clear()
date_from.send_keys("01/01/2021")
date_to.clear()
date_to.send_keys("01/03/2024")

print("Clicking the search button...")
search_button_xpath = '//*[@id="ctl00_Content_btnSearch"]'
search_button = wait.until(EC.element_to_be_clickable((By.XPATH, search_button_xpath)))
search_button.click()

print("Starting to scrape data...")
results = []
current_page = 1

while True:
    try:
        print(f"Scraping data from page {current_page}...")
        wait.until(EC.visibility_of_element_located((By.XPATH, '//*[@id="ctl00_Content_cusResultsGrid_repWebGrid_ctl00_grdWebGridTabularView"]')))
        rows = driver.find_elements(By.XPATH, '//*[@id="ctl00_Content_cusResultsGrid_repWebGrid_ctl00_grdWebGridTabularView"]/tbody/tr')

        for row in rows[1:-1]:  # Skipping the header and pagination rows
            cells = row.find_elements(By.TAG_NAME, "td")
            if len(cells) >= 7:  # Ensure there are enough cells
                link = cells[0].find_element(By.TAG_NAME, "a").text if cells[0].find_elements(By.TAG_NAME, "a") else 'No Link'
                date = cells[1].text
                proposal = cells[2].text
                app_type = cells[3].text
                address = cells[5].text
                status = cells[6].text
                results.append([link, date, proposal, app_type, address, status])

        # Handle pagination
        next_page_number = current_page + 1
        if current_page % 10 == 0:
            print(f"Handling special pagination at page {current_page}...")
            ellipsis_xpath = f'//*[@id="ctl00_Content_cusResultsGrid_repWebGrid_ctl00_grdWebGridTabularView"]/tbody/tr[17]/td/table/tbody/tr/td[{11 if current_page == 10 else 12}]/a'
            try:
                driver.find_element(By.XPATH, ellipsis_xpath).click()
                print(f"Clicked on '...' to navigate to special page after page {current_page}.")
                time.sleep(3)  # Wait for the next set of pages to load
            except NoSuchElementException:
                print("No '...' link found. Exiting loop.")
                break  # Exit loop if '...' link is not found
        else:
            next_page_xpath = f'//*[@id="ctl00_Content_cusResultsGrid_repWebGrid_ctl00_grdWebGridTabularView"]/tbody/tr[17]/td/table/tbody/tr/td/a[contains(text(), "{next_page_number}")]'
            try:
                driver.find_element(By.XPATH, next_page_xpath).click()
                print(f"Navigating to page {next_page_number}.")
                time.sleep(3)  # Wait for the next page to load
            except NoSuchElementException:
                print(f"No link found for page {next_page_number}. Exiting loop.")
                break  # Exit loop if next page number is not found

        current_page += 1

    except TimeoutException:
        print("Timeout exception occurred. Exiting loop.")
        break  # Exit the loop if the table is not visible

print("Closing the browser...")
driver.quit()

print("Saving results to CSV file...")
df = pd.DataFrame(results, columns=['Link', 'Date', 'Proposal', 'Type', 'Address', 'Status'])
df.to_csv('planning_applications.csv', index=False)
print("Data scraping complete. File 'planning_applications.csv' saved.")
